In [18]:
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import numpy as np
import warnings
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
warnings.filterwarnings('ignore')

In [6]:
base_dir='./dataSet/'
train_path=os.path.join(base_dir,'trainSet')
test_path=os.path.join(base_dir,'validationSet')
forest_path=os.path.join(base_dir,'testSet')

In [16]:
a=Image.open('./dataSet/trainSet/AKIEC/ISIC_0024329.jpg')
np.array(a).shape

(450, 600, 3)

In [11]:
os.listdir(train_path)

['AKIEC', '.DS_Store', 'DF', 'BKL', 'VASC', 'NV', 'BCC', 'MEL']

In [25]:
model=tf.keras.Sequential()

##layer 1
# model.add(tf.keras.layers.Input())
model.add(tf.keras.layers.Conv2D(32,(3,3),input_shape=(450, 600, 3)))
# model.add(tf.keras.layers.BatchNormalization())##把激活的数据做归一化，在激活函数前面
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.MaxPool2D(2,2,padding='SAME'))


##layer2
model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(tf.keras.layers.MaxPool2D(2,2,padding='SAME'))

##faltten,多维向低维的转换
model.add(tf.keras.layers.Flatten())

##全连阶层
model.add(tf.keras.layers.Dense(256,activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(7,activation='softmax'))


model.compile(optimizer='adam',metrics=['acc'],loss=['categorical_crossentropy'])

print(model.summary())


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 448, 598, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 448, 598, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 224, 299, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 222, 297, 64)      18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 111, 149, 64)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1058496)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)              

In [26]:
###数据的预处理--读进来的数据会被自动转换为tensor格式，分别准备训练和测试
#图像的归一化（0-1）之间
train_dataGen=ImageDataGenerator(rescale=1./255)
test_dataGen=ImageDataGenerator(rescale=1./255)
forest_dataGen=ImageDataGenerator(rescale=1./255)

In [27]:
##从文件夹中读取数据
train_generator=train_dataGen.flow_from_directory(train_path,#训练数据路径
                                                  target_size=(450,600),#制定resize大小,和输入大小想匹配
                                                  batch_size=20,##一次拿出20张图片来训练
                                                  class_mode='categorical'##和标签相关，one-hot的话就用categorical，二分类的话可以用binary
                                                  )

test_generator=train_dataGen.flow_from_directory(test_path,#训练数据路径
                                                  target_size=(450,600),#制定resize大小
                                                  batch_size=20,
                                                  class_mode='categorical'##one-hot的话就用categorical，二分类的话可以用binary
                                                  )

Found 7004 images belonging to 7 classes.
Found 997 images belonging to 7 classes.


In [28]:
##训练网络模型
#直接fit也是可以的，但是通常不能把所有数据全部放入内存,因为有时候数据特别特别大，fit_generator相当于一个生成器，动态产生所需的batch数据，训练的时候用谁，拿谁，不用全部都读到内存中去
#step_per_epoch 相当于给定一个停止条件，因为生成器会不断的产生batch数据，也就是说他不知道一个epoch里需要执行多少个step

history=model.fit_generator(train_generator,
                            steps_per_epoch=100,#steps_per_epoch*20=2000
                            epochs=20,#每个batch训练迭代多少次
                            validation_data=test_generator,##需不需要做验证
                            verbose=2,
                            validation_steps=50#validation_steps*20=10000
                            )

Epoch 1/20


KeyboardInterrupt: 